# functools 增强函数功能

partial(func, *args, **keywords)
> 创建偏函数：将某些不变的参数冻结，适用于某次使用只需要更改部分参数的情况

wraps
> 使用装饰器修饰后的函数会丢失元信息（如名字、文档字符串、注解和参数签名），导致在多进程中不可被pickle，而使用wraps充当其装饰器时，可以避免这个问题

In [4]:
from functools import partial,wraps

In [1]:
def take(n, iterable):
    """Return first *n* items of the iterable as a list.
        >>> take(3, range(10))
        [0, 1, 2]
    """
    return list(itertools.islice(iterable, n))

def chunked(iterable, n):
    """Break *iterable* into lists of length *n*:
        >>> list(chunked([1, 2, 3, 4, 5, 6], 3))
        [[1, 2, 3], [4, 5, 6]]
        >>> list(chunked([1, 2, 3, 4, 5, 6, 7, 8], 3))
        [[1, 2, 3], [4, 5, 6], [7, 8]]
    """
    return iter(functools.partial(take, n, iter(iterable)), [])  

#iter(callable, sentinel) 会持续调用参数callable直至其返回sentinel，因此会使得iter(iterable)不断进行直至抛出错误

In [2]:
def my_decorator(func):  
    @wraps
    def wrapper(*args, **kwargs):  
        print('Calling decorated function...')  
        return func(*args, **kwargs)  
    return wrapper  

#  itertools 各种迭代器 

In [3]:
import itertools

## 无限迭代 
count(start=0, step=1) --> start, start+step, start+2 * step, ... 
> count(10) --> 10 11 12 13 14 ...

cycle(p) --> p0, p1, ... plast, p0, p1, ...
> cycle('ABCD') --> A B C D A B C D ...

repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
> repeat(10, 3) --> 10 10 10

## 有限迭代
accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
> accumulate([1,2,3,4,5]) --> 1 3 6 10 15

chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ... 
> chain('ABC', 'DEF') --> A B C D E F

compress(iterable, Boolean)：输出iterable对象中对应选择器为True的元素

dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails

### accumulate(p[, func]) 
> 累加：p0, p0+p1, p0+p1+p2

也可以做其它用处

In [7]:
def accumulate(iterable, func=operator.add):
    it = iter(iterable)
    try:
        total = next(it)
    except StopIteration:
        return
    yield total
    for element in it:
        total = func(total, element)
        yield total

In [8]:
_iter = itertools.accumulate([1,2,3,4,5])
list(_iter)

[1, 3, 6, 10, 15]

In [9]:
_iter = itertools.accumulate([1,2,3,4,5],operator.mul)
list(_iter)

[1, 2, 6, 24, 120]

In [10]:
_iter = itertools.accumulate([1,2,3,4,5],max)
list(_iter)

[1, 2, 3, 4, 5]

In [11]:
_iter = itertools.accumulate([1,2,3,4,5],lambda bal, pmt: bal*2 + pmt)
list(_iter)

[1, 4, 11, 26, 57]

### chain(p, q, ...) 
> 把多个迭代体合并：p0, p1, ... plast, q0, q1, ... 

In [12]:
def chain(*iterables):
    for it in iterables:
        for element in it:
            yield element

In [13]:
_iter = itertools.chain('ABC', 'DEF')
list(_iter)

['A', 'B', 'C', 'D', 'E', 'F']

### compress(iterable, Boolean)
> 输出iterable对象中对应选择器为True的元素

In [15]:
addresses = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
]
_bool = [False, False, True, False, False, True, True, False]
_iter = itertools.compress(addresses, _bool)
list(_iter)

['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']

### dropwhile(pred, seq) 
> 到某处开始：seq[n], seq[n+1], starting when pred fails 

In [16]:
def dropwhile(predicate, iterable):
    iterable = iter(iterable)
    for x in iterable:
        if not predicate(x):
            yield x
            break
    for x in iterable:
        yield x

In [17]:
_iter = itertools.dropwhile(lambda x: x<5, [1,4,6,4,1])
list(_iter)

[6, 4, 1]

### takewhile(pred, seq) 
> 到某处停止：seq[0], seq[1], until pred fails 

In [18]:
_iter = itertools.takewhile(lambda x: x<5, [1,4,6,4,1])
list(_iter)

[1, 4]

### filterfalse(pred, seq) 
> 根据条件筛选：filter elements of seq where pred(elem) is False

In [19]:
_iter = itertools.filterfalse(lambda x: x%2, range(10)) 
list(_iter)

[0, 2, 4, 6, 8]

### islice(seq, [start,] stop [, step]) 
> 切片：elements from seq[start:stop:step] 

In [20]:
def islice(iterable, *args):
    s = slice(*args)
    it = iter(range(s.start or 0, s.stop or sys.maxsize, s.step or 1))
    try:
        nexti = next(it)
    except StopIteration:
        return
    for i, element in enumerate(iterable):
        if i == nexti:
            yield element
            nexti = next(it)

In [21]:
_iter = itertools.islice('ABCDEFG', 2, None) 
list(_iter)

['C', 'D', 'E', 'F', 'G']

### starmap(fun, seq) 
> map升级版：fun(*seq[0]), fun(*seq[1]) 

In [22]:
_iter = itertools.starmap(pow, [(2,5), (3,2), (10,3)])
list(_iter)

[32, 9, 1000]

### tee(it, n=2) 
> 分割迭代器：(it1, it2 , ... itn)

In [23]:
def tee(iterable, n=2):
    it = iter(iterable)
    deques = [collections.deque() for i in range(n)]
    def gen(mydeque):
        while True:
            if not mydeque:             # when the local deque is empty
                try:
                    newval = next(it)   # fetch a new value and
                except StopIteration:
                    return
                for d in deques:        # load it to all the deques
                    d.append(newval)
            yield mydeque.popleft()
    return tuple(gen(d) for d in deques)

### zip_longest(p, q, ...) 
> 按最长长度zip：(p[0], q[0]), (p[1], q[1]), ...  

In [24]:
def zip_longest(*args, **kwds):
    # zip_longest('ABCD', 'xy', fillvalue='-') --> Ax By C- D-
    fillvalue = kwds.get('fillvalue')
    counter = len(args) - 1
    def sentinel():
        nonlocal counter
        if not counter:
            raise ZipExhausted
        counter -= 1
        yield fillvalue
    fillers = repeat(fillvalue)
    iterators = [chain(it, sentinel(), fillers) for it in args]
    try:
        while iterators:
            yield tuple(map(next, iterators))
    except ZipExhausted:
        pass

In [25]:
_iter = itertools.zip_longest('ABCD', 'xy', fillvalue='-')
list(_iter)

[('A', 'x'), ('B', 'y'), ('C', '-'), ('D', '-')]

### 排列组合 
> product(p, q, ... [repeat=1])：有放回的排列

> permutations(p[, r])：排列

> combinations(p, r)：组合

> combinations_with_replacement(p, r) ：有放回的组合

In [26]:
def permutations(iterable, r=None):
    # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    if r > n:
        return
    indices = list(range(n))
    cycles = list(range(n, n-r, -1))
    yield tuple(pool[i] for i in indices[:r])
    while n:
        for i in reversed(range(r)):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in indices[:r])
                break
        else:
            return

In [27]:
def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [28]:
def combinations_with_replacement(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if not n and r:
        return
    indices = [0] * r
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != n - 1:
                break
        else:
            return
        indices[i:] = [indices[i] + 1] * (r - i)
        yield tuple(pool[i] for i in indices)

In [29]:
_iter = itertools.product('ABCD', repeat=2)
list(_iter)

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C'),
 ('C', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('D', 'C'),
 ('D', 'D')]

In [30]:
_iter = itertools.permutations('ABCD',2)
list(_iter)

[('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'A'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('D', 'C')]

In [31]:
_iter = itertools.combinations('ABCD',2)
list(_iter)

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]

In [32]:
_iter = itertools.combinations_with_replacement('ABCD',2)
list(_iter)

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'C'),
 ('C', 'D'),
 ('D', 'D')]

# operator 内置操作符接口

In [6]:
import operator

# more_itertools
itertools模块的补充

In [38]:
import more_itertools

## chunked(iterable, n) 
将迭代器的每n项提出来组成一个新的list

等同于 zip(\*[iter(iterable)]\*n)

In [39]:
def take(n, iterable):
    """Return first *n* items of the iterable as a list.
        >>> take(3, range(10))
        [0, 1, 2]
    """
    return list(itertools.islice(iterable, n))

def chunked(iterable, n):
    """Break *iterable* into lists of length *n*:
        >>> list(chunked([1, 2, 3, 4, 5, 6], 3))
        [[1, 2, 3], [4, 5, 6]]
        >>> list(chunked([1, 2, 3, 4, 5, 6, 7, 8], 3))
        [[1, 2, 3], [4, 5, 6], [7, 8]]
    """
    return iter(functools.partial(take, n, iter(iterable)), []) 

## collate(*iterables)     	
按序整合：比itertools.chain更进一步

In [40]:
def collapse(iterable, base_type=None, levels=None):
    """
    Flatten an iterable with multiple levels of nesting into non-iterable types.
    >>> iterable = [(1, 2), ([3, 4], [[5], [6]])]
    >>> list(collapse(iterable))
            [1, 2, 3, 4, 5, 6]
    String types are not considered iterable and will not be collapsed.
    To avoid collapsing other types, specify *base_type*:
    >>> iterable = ['ab', ('cd', 'ef'), ['gh', 'ij']]
    >>> list(collapse(iterable, base_type=tuple))
            ['ab', ('cd', 'ef'), 'gh', 'ij']
    Specify *levels* to stop flattening after a certain level:
    >>> iterable = [('a', ['b']), ('c', ['d'])]
    >>> list(collapse(iterable, levels=1))  # Only one level flattened
        ['a', ['b'], 'c', ['d']]
    """
    def walk(node, level):
        if (
            ((levels is not None) and (level > levels)) or
            isinstance(node, string_types) or
            ((base_type is not None) and isinstance(node, base_type))
        ):
            yield node
            return

        try:
            tree = iter(node)
        except TypeError:
            yield node
            return
        else:
            for child in tree:
                for x in walk(child, level + 1):
                    yield x
    for x in walk(iterable, 0):
        yield x

In [41]:
_iter = more_itertools.collate('ACDZ', 'AZ', 'JKL')
list(_iter)

['A', 'A', 'C', 'D', 'J', 'K', 'L', 'Z', 'Z']

## distribute(n, iterable)
distribute the items from iterable among n smaller iterables

In [42]:
children = more_itertools.distribute(3, [1, 2, 3, 4, 5, 6, 7])
[list(c) for c in children]

[[1, 4, 7], [2, 5], [3, 6]]

## split_at(iterable, pred) 
The lists do not include the delimiting items

In [43]:
list(more_itertools.split_at(range(10), lambda n: n % 2 == 1))

[[0], [2], [4], [6], [8], []]

## split_before(iterable, pred)
each list starts with an item where callable pred returns 

In [44]:
list(more_itertools.split_before(range(10), lambda n: n % 3 == 0))

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]

## windowed(seq, n, fillvalue=None, step=1)
滑动

In [45]:
all_windows = more_itertools.windowed([1, 2, 3, 4, 5], 3)
list(all_windows)

[(1, 2, 3), (2, 3, 4), (3, 4, 5)]

## count_cycle(iterable, n=None)
Cycle through the items from iterable up to n times

相当于iterable和range(n)之间的组合

In [46]:
list(more_itertools.count_cycle('AB', 3))

[(0, 'A'), (0, 'B'), (1, 'A'), (1, 'B'), (2, 'A'), (2, 'B')]

## intersperse(e, iterable, n=1)
Intersperse filler element e among the items in iterable, leaving n items between each filler element

In [47]:
list(more_itertools.intersperse('!', [1, 2, 3, 4, 5], n=2))

[1, 2, '!', 3, 4, '!', 5]